<a href="https://colab.research.google.com/github/t-abs/Autonomous-cars-IISER-Intern/blob/main/Motion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sample code for motion detect

In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Define a custom dataset class for motion detection
class MotionDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.frames = os.listdir(os.path.join(data_dir, "frames"))
        self.labels = np.load(os.path.join(data_dir, "labels.npy"))

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, "frames", self.frames[idx])
        image = cv2.imread(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define a simple CNN model for motion detection
class MotionDetectionModel(nn.Module):
    def __init__(self):
        super(MotionDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 75 * 100, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 75 * 100)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.sigmoid(self.fc2(x))
        return x

# Define training parameters
data_dir = "https://example.com/dataset"  # Replace with actual URL to your dataset
batch_size = 32
lr = 0.001
epochs = 10

# Create dataset and data loader
dataset = MotionDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = MotionDetectionModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(epochs):
    for images, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Save trained model
torch.save(model.state_dict(), "motion_detection_model.pth")
